# Sysmon JSONL to CSV Converter

## 📖 Overview

This notebook transforms Windows Sysmon events from JSONL format (extracted from Elasticsearch) into structured CSV datasets for machine learning analysis. It processes raw Windows Event Log XML embedded within JSONL files and converts them into tabular format with proper field mapping per event type.

### 🎯 Purpose

- **Parse** Windows Event Log XML from JSONL files  
- **Extract** structured data using event-specific field schemas
- **Transform** multi-format security events into unified CSV structure
- **Clean** and validate data for machine learning pipeline readiness

### 📊 Input/Output

- **Input**: JSONL file with Windows Sysmon events (from notebook #1)
- **Output**: Structured CSV file with event-specific columns and proper field mapping

### 🔧 Key Features

- **Schema-based parsing** for 18+ different Sysmon event types
- **Robust XML handling** with sanitization and error recovery
- **Detailed logging** for debugging and quality assessment
- **Field mapping validation** with missing field tracking

---

## 🛠️ Required Libraries

Import essential libraries for XML parsing, data manipulation, and file processing:

In [10]:
import json
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd

## 📋 Sysmon Event Schema Mapping

Define field mappings for each Sysmon EventID based on official documentation and data analysis:

In [11]:
# Fields obtained from https://www.ultimatewindowssecurity.com/securitylog/encyclopedia/ and manual inspection of the data
# EventID 8 uses lowercase 'guid' in XML, but we'll map to uppercase 'GUID' columns in DataFrame for consistency
fields_per_eventid = {
1: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'CommandLine', 'CurrentDirectory', 'User', 'Hashes', 'ParentProcessGuid', 'ParentProcessId', 'ParentImage', 'ParentCommandLine'],
2: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetFilename', 'CreationUtcTime', 'PreviousCreationUtcTime', 'User'],
3: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'User', 'Protocol', 'SourceIsIpv6', 'SourceIp', 'SourceHostname', 'SourcePort', 'SourcePortName', 'DestinationIsIpv6', 'DestinationIp', 'DestinationHostname', 'DestinationPort', 'DestinationPortName'],
5: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'User'],
6: ['UtcTime', 'ImageLoaded', 'Hashes', 'User'],
7: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'ImageLoaded', 'OriginalFileName', 'Hashes', 'User'],
8: ['UtcTime', 'SourceProcessGuid', 'SourceProcessId', 'SourceImage', 'TargetProcessGuid', 'TargetProcessId', 'TargetImage', 'NewThreadId', 'SourceUser', 'TargetUser'],
9: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'Device', 'User'],
10: ['UtcTime', 'SourceProcessGUID', 'SourceProcessId', 'SourceImage', 'TargetProcessGUID', 'TargetProcessId', 'TargetImage', 'SourceThreadId', 'SourceUser', 'TargetUser'],
11: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetFilename', 'CreationUtcTime', 'User'],
12: ['EventType', 'UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetObject', 'User'],
13: ['EventType', 'UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetObject', 'User'],
14: ['EventType', 'UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetObject', 'User'],
15: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'TargetFilename', 'CreationUtcTime', 'Hash', 'User'],
16: ['UtcTime', 'Configuration', 'ConfigurationFileHash', 'User'],
17: ['EventType', 'UtcTime', 'ProcessGuid', 'ProcessId', 'PipeName', 'Image', 'User'],
18: ['EventType', 'UtcTime', 'ProcessGuid', 'ProcessId', 'PipeName', 'Image', 'User'],
22: ['UtcTime', 'ProcessGuid', 'ProcessId', 'Image', 'QueryName', 'QueryStatus', 'QueryResults', 'User'],
23: ['UtcTime', 'ProcessGuid', 'ProcessId', 'User', 'Image', 'TargetFilename', 'Hashes'],
24: ['UtcTime', 'ProcessGuid', 'ProcessId', 'User', 'Image', 'Hashes'],
25: ['UtcTime', 'ProcessGuid', 'ProcessId', 'User', 'Image']
}

## 🧹 XML Sanitization Function

Clean and repair malformed XML strings before parsing:

In [12]:
def sanitize_xml(xml_str):
    """Clean invalid characters and repair XML structure"""
    # Remove non-printable characters
    cleaned = ''.join(c for c in xml_str if 31 < ord(c) < 127 or c in '\t\n\r')
    # Fix common XML issues using BeautifulSoup's parser
    return BeautifulSoup(cleaned, "xml").prettify()

## 🔍 XML Parser Function

Parse Windows Event Log XML to extract EventID, Computer, and event-specific fields:

In [13]:
def parse_sysmon_event(xml_str):
    """Parse XML with enhanced error handling"""
    try:
        # Clean XML first
        clean_xml = sanitize_xml(xml_str)
        
        # Parse with explicit namespace
        namespaces = {'ns': 'http://schemas.microsoft.com/win/2004/08/events/event'}
        root = ET.fromstring(clean_xml)
        
        # System section - with null checks
        system = root.find('ns:System', namespaces)
        if not system:
            return None, None, None

        event_id_elem = system.find('ns:EventID', namespaces)
        computer_elem = system.find('ns:Computer', namespaces)
        
        # event_id = event_id_elem.text.strip() if (event_id_elem and event_id_elem.text) else None
        event_id = int(event_id_elem.text) if event_id_elem is not None else None
        
        # computer = computer_elem.text.strip() if (computer_elem and computer_elem.text) else None
        computer = computer_elem.text if computer_elem is not None else None

        # EventData section
        event_data = root.find('ns:EventData', namespaces)
        fields = {}
        if event_data:
            for data in event_data.findall('ns:Data', namespaces):
                name = data.get('Name')
                # fields[name] = data.text.strip() if data.text else None
                fields[name] = data.text if data.text else None

        return event_id, computer, fields

    except Exception as e:
        # Log problematic XML samples for debugging
        with open('bad_xml_samples.txt', 'a') as bad_xml:
            bad_xml.write(f"Error: {str(e)}\n")
            bad_xml.write(f"XML: {xml_str[:500]}...\n")
            bad_xml.write("-" * 50 + "\n")
        print(f"XML parsing failed: {str(e)}")
        return None, None, None  # Crucial: return tuple of 3 Nones

## 🔧 Data Type Conversion Helper Functions

Define utility functions for proper data type conversion and cleaning:

In [14]:
def safe_int_conversion(value):
    """Safely convert value to integer, handling whitespace, NaN and invalid values"""
    if value is None or pd.isna(value):
        return None
    try:
        # Strip whitespace first, then convert
        cleaned_value = str(value).strip()
        if not cleaned_value:
            return None
        return int(float(cleaned_value))
    except (ValueError, TypeError):
        return None

def clean_guid(value):
    """Remove whitespace and brackets from GUID values and ensure string type"""
    if value is None or pd.isna(value):
        return None
    try:
        # First strip whitespace, then remove curly brackets
        cleaned = str(value).strip()
        if not cleaned:
            return None
        # Remove curly brackets
        cleaned = cleaned.strip('{}')
        return cleaned if cleaned else None
    except (ValueError, TypeError):
        return None

## ⚙️ Main Processing Function

Process JSONL file and convert Sysmon events to structured DataFrame with comprehensive error tracking:

In [15]:
def process_events(jsonl_path):
    """Process events with error tracking and data type optimization"""
    records = []
    error_count = 0
    missing_fields_tracker = {}  # Track missing fields per EventID
    eventid_counts = {}  # Track total events per EventID
    
    # Define columns that should be integers
    integer_columns = {
        'ProcessId', 'SourcePort', 'DestinationPort', 'SourceProcessId', 
        'ParentProcessId', 'SourceThreadId', 'TargetProcessId'
    }
    
    # Define columns that are GUIDs (need bracket stripping)
    guid_columns = {
        'ProcessGuid', 'SourceProcessGUID', 'TargetProcessGUID', 'ParentProcessGuid'
    }
    
    with open(jsonl_path, 'r') as f:
        for line_number, line in enumerate(f, 1):
            try:
                event = json.loads(line)
                xml_str = event['event']['original']
                
                event_id, computer, fields = parse_sysmon_event(xml_str)
                
                # Skip if essential fields missing with detailed logging
                if not event_id or not computer:
                    with open('parsing_errors.log', 'a') as log:
                        log.write(f"Line {line_number}: Failed - EventID={event_id}, Computer={computer}\n")
                        log.write(f"XML sample: {xml_str[:200]}...\n\n")
                    error_count += 1
                    continue
                
                # Count total events per EventID
                eventid_counts[event_id] = eventid_counts.get(event_id, 0) + 1
                
                # Build record - REMOVED @timestamp field to avoid redundancy with UtcTime
                record = {
                    'EventID': event_id,
                    'Computer': computer
                }
                
                # Add fields from mapping with mismatch tracking and data type optimization
                expected_fields = fields_per_eventid.get(event_id, [])
                for field in expected_fields:
                    # Special handling for EventID 8: map lowercase guid fields to uppercase GUID columns
                    if event_id == 8:
                        if field == 'SourceProcessGuid':
                            # Store in uppercase GUID column for consistency with EventID 10
                            column_name = 'SourceProcessGUID'
                            if field not in fields:
                                if event_id not in missing_fields_tracker:
                                    missing_fields_tracker[event_id] = {}
                                if field not in missing_fields_tracker[event_id]:
                                    missing_fields_tracker[event_id][field] = 0
                                missing_fields_tracker[event_id][field] += 1
                            
                            # Get value and apply GUID cleaning
                            raw_value = fields.get(field, None)
                            record[column_name] = clean_guid(raw_value)
                            continue
                        elif field == 'TargetProcessGuid':
                            # Store in uppercase GUID column for consistency with EventID 10
                            column_name = 'TargetProcessGUID'
                            if field not in fields:
                                if event_id not in missing_fields_tracker:
                                    missing_fields_tracker[event_id] = {}
                                if field not in missing_fields_tracker[event_id]:
                                    missing_fields_tracker[event_id][field] = 0
                                missing_fields_tracker[event_id][field] += 1
                            
                            # Get value and apply GUID cleaning
                            raw_value = fields.get(field, None)
                            record[column_name] = clean_guid(raw_value)
                            continue
                    
                    # Normal field processing for all other cases
                    if field not in fields:  # Field expected but not found in XML
                        if event_id not in missing_fields_tracker:
                            missing_fields_tracker[event_id] = {}
                        if field not in missing_fields_tracker[event_id]:
                            missing_fields_tracker[event_id][field] = 0
                        missing_fields_tracker[event_id][field] += 1
                    
                    # Get the raw value
                    raw_value = fields.get(field, None)
                    
                    # Apply data type conversions based on field name
                    if field in integer_columns:
                        # Convert to integer using module-level function
                        record[field] = safe_int_conversion(raw_value)
                    elif field in guid_columns:
                        # Clean GUID using module-level function
                        record[field] = clean_guid(raw_value)
                    else:
                        # Keep as-is for other fields but strip whitespace if it's a string
                        if raw_value is not None:
                            cleaned_value = str(raw_value).strip()
                            record[field] = cleaned_value if cleaned_value else None
                        else:
                            record[field] = raw_value
                
                records.append(record)
                
            except Exception as e:
                print(f"Error processing line {line_number}: {str(e)}")
                error_count += 1
    
    print(f"\nProcessing complete. Errors: {error_count}, Success: {len(records)}")
    
    # Print detailed missing fields summary with statistics
    if missing_fields_tracker:
        print("\nDetailed missing fields analysis:")
        print("-" * 60)
        for event_id, missing_fields in missing_fields_tracker.items():
            total_events = eventid_counts[event_id]
            print(f"EventID {event_id}: {total_events} total events")
            for field, missing_count in missing_fields.items():
                percentage = (missing_count / total_events) * 100
                print(f"  • Field '{field}': {missing_count}/{total_events} missing ({percentage:.1f}%)")
            print()
    else:
        print("\nNo missing fields detected - all schema mappings match XML structure.")
    
    # Print EventID distribution summary
    print("EventID distribution:")
    print("-" * 40)
    for event_id in sorted(eventid_counts.keys()):
        print(f"EventID {event_id}: {eventid_counts[event_id]:,} events")
    
    return pd.DataFrame(records)

## 📁 File Configuration

Set input and output file paths for processing:

In [16]:
filepath = "-ds-logs-windows-sysmon_operational-default-2025-05-04-000001.jsonl"
output_file = "sysmon-2025-05-04-000001.csv"

In [19]:
df = process_events(filepath)


Processing complete. Errors: 0, Success: 570078

Detailed missing fields analysis:
------------------------------------------------------------
EventID 6: 481 total events
  • Field 'User': 481/481 missing (100.0%)

EventID distribution:
----------------------------------------
EventID 1: 1,461 events
EventID 2: 57 events
EventID 3: 16,918 events
EventID 4: 6 events
EventID 5: 965 events
EventID 6: 481 events
EventID 7: 63,892 events
EventID 9: 1,158 events
EventID 10: 57,814 events
EventID 11: 4,271 events
EventID 12: 289,812 events
EventID 13: 129,227 events
EventID 15: 46 events
EventID 17: 488 events
EventID 18: 1,491 events
EventID 23: 1,979 events
EventID 24: 5 events
EventID 25: 7 events


## 🧽 Data Cleaning Function

Clean DataFrame by removing whitespace and normalizing empty values:

In [20]:
def clean_dataframe(df):
    """Clean whitespace and optimize data types"""
    # Trim whitespace for string columns
    str_cols = df.select_dtypes(['object']).columns
    df[str_cols] = df[str_cols].apply(lambda x: x.str.strip())
    
    # Replace empty strings with None
    df.replace({'': None}, inplace=True)
    
    # Optimize data types for better performance and storage
    
    # Integer columns - convert to nullable integer type
    integer_columns = ['ProcessId', 'SourcePort', 'DestinationPort', 'SourceProcessId', 
                      'ParentProcessId', 'SourceThreadId', 'TargetProcessId']
    
    for col in integer_columns:
        if col in df.columns:
            # Convert to nullable integer type (Int64 handles NaN properly)
            df[col] = df[col].astype('Int64')
            print(f"Converted {col} to Int64 type")
    
    # GUID columns - ensure they're string type (not object)
    guid_columns = ['ProcessGuid', 'SourceProcessGUID', 'TargetProcessGUID', 'ParentProcessGuid']
    
    for col in guid_columns:
        if col in df.columns:
            # Convert to string type (handles NaN as <NA>)
            df[col] = df[col].astype('string')
            print(f"Converted {col} to string type")
    
    # Convert Computer and other categorical-like columns to category type for memory efficiency
    categorical_columns = ['Computer', 'Protocol', 'EventType']
    
    for col in categorical_columns:
        if col in df.columns and df[col].nunique() < df.shape[0] * 0.5:  # Only if less than 50% unique values
            df[col] = df[col].astype('category')
            print(f"Converted {col} to category type")
    
    print(f"\nData type optimization complete!")
    print(f"Memory usage after optimization: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return df

## 🧼 Apply Data Cleaning

Clean the processed DataFrame to prepare for CSV export:

In [21]:
df = clean_dataframe(df)

Converted ProcessId to Int64 type
Converted SourcePort to Int64 type
Converted DestinationPort to Int64 type
Converted SourceProcessId to Int64 type
Converted ParentProcessId to Int64 type
Converted SourceThreadId to Int64 type
Converted TargetProcessId to Int64 type
Converted ProcessGuid to string type
Converted SourceProcessGUID to string type
Converted TargetProcessGUID to string type
Converted ParentProcessGuid to string type
Converted Computer to category type
Converted Protocol to category type
Converted EventType to category type

Data type optimization complete!
Memory usage after optimization: 847.15 MB


## 💾 Export to CSV

Save the cleaned DataFrame as CSV file for machine learning pipeline:

In [22]:
df.to_csv(output_file, index=False)

## 🔍 Optional Data Exploration

The following commented cells provide various data exploration and validation options:

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570078 entries, 0 to 570077
Data columns (total 44 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   EventID                  570078 non-null  int64   
 1   Computer                 570078 non-null  category
 2   UtcTime                  570072 non-null  object  
 3   ProcessGuid              511777 non-null  string  
 4   ProcessId                511777 non-null  Int64   
 5   Image                    511777 non-null  object  
 6   User                     511770 non-null  object  
 7   Protocol                 16918 non-null   category
 8   SourceIsIpv6             16918 non-null   object  
 9   SourceIp                 16918 non-null   object  
 10  SourceHostname           16918 non-null   object  
 11  SourcePort               16918 non-null   Int64   
 12  SourcePortName           16918 non-null   object  
 13  DestinationIsIpv6        16918 non-null   ob

In [11]:
# import pandas as pd

In [12]:
# new_df = pd.read_csv(output_file)

In [13]:
# new_df.head()

In [14]:
# new_df.info()

In [15]:
# # check rows where the Utctime is null
# new_df[new_df['UtcTime'].isnull()].info()

In [16]:
# new_df[new_df['UtcTime'].isnull()]['EventID'].unique()

In [17]:
# new_df['EventID'].unique()

In [18]:
# new_df[new_df['EventID'] == 1].head(2)

In [19]:
# new_df[new_df['EventID'] == 3].head(2)

In [20]:
# new_df[new_df['EventID'] == 5].head(2)

In [21]:
# new_df[new_df['EventID'] == 6].head(2)

In [22]:
# new_df[new_df['EventID'] == 6]['ImageLoaded'].unique()

In [23]:
# new_df[new_df['EventID'] == 7].head(2)

In [24]:
# new_df[new_df['EventID'] == 8].head(2)

In [25]:
# new_df[new_df['EventID'] == 9].head(2)

In [26]:
# new_df[new_df['EventID'] == 10].head(2)

In [27]:
# new_df[new_df['EventID'] == 10]['TargetProcessGUID'].unique()

In [28]:
# new_df[new_df['EventID'] == 11].head(2)

In [29]:
# new_df[new_df['EventID'] == 12].head(2)

In [30]:
# new_df[new_df['EventID'] == 13].head(2)

In [31]:
# new_df[new_df['EventID'] == 14].head(2)

In [32]:
# new_df[new_df['EventID'] == 15].head(2)

In [33]:
# new_df[new_df['EventID'] == 16].head(2)

In [34]:
# new_df[new_df['EventID'] == 17].head(2)

In [35]:
# new_df[new_df['EventID'] == 18].head(2)

In [36]:
# new_df[new_df['EventID'] == 22].head(2)

In [37]:
# new_df[new_df['EventID'] == 23].head(2)

In [38]:
# new_df[new_df['EventID'] == 24].head(2)

In [39]:
# new_df[new_df['EventID'] == 25].head(2)